**This is example of how to trace model with jit and export it to the onnx**

In [1]:
from rl_games.torch_runner import Runner
import ray
import yaml
import torch
import matplotlib.pyplot as plt
import gym
from IPython import display
import numpy as np
import onnx
import onnxruntime as ort
%matplotlib inline

/home/viktorm/anaconda3/envs/warp39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/viktorm/anaconda3/envs/warp39/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):


In [2]:
ray.init(object_store_memory=1024*1024*1000)

/home/viktorm/anaconda3/envs/warp39/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


RayContext(dashboard_url='', python_version='3.9.12', ray_version='1.12.0', ray_commit='f18fc31c7562990955556899090f8e8656b48d2d', address_info={'node_ip_address': '10.2.168.57', 'raylet_ip_address': '10.2.168.57', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-09-16_13-05-39_286634_1198493/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-09-16_13-05-39_286634_1198493/sockets/raylet', 'webui_url': '', 'session_dir': '/tmp/ray/session_2022-09-16_13-05-39_286634_1198493', 'metrics_export_port': 50541, 'gcs_address': '10.2.168.57:55676', 'address': '10.2.168.57:55676', 'node_id': '90e2799f4b795ea1506755e4e3edf1b9fc7df173b9563c45bc412b9d'})

In [3]:
config_name = '../rl_games/configs/ppo_cartpole.yaml'

In [ ]:
with open(config_name, 'r') as stream:
    config = yaml.safe_load(stream)
    config['params']['config']['normalize_input'] = True
    config['params']['config']['max_epochs'] = 1000
    config['params']['config']['full_experiment_name'] = 'cartpole_onnx'
runner = Runner()
runner.load(config)
runner.run({
    'train': True,
})

self.seed = 1663358741
Started to train
Exact experiment name requested from command line: cartpole_onnx
current training device: cuda
build mlp: 4
build mlp: 4
RunningMeanStd:  (4,)


/home/viktorm/anaconda3/envs/warp39/lib/python3.9/site-packages/rl_games/common/experience.py:341: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.tensor_dict['actions'] = self._create_tensor_from_space(gym.spaces.Box(low=0, high=1,shape=self.actions_shape, dtype=np.long), obs_base_shape)


fps step: 3837 fps step and policy inference: 611 fps total: 537 epoch: 1/1000
fps step: 3651 fps step and policy inference: 2651 fps total: 1677 epoch: 2/1000
fps step: 3597 fps step and policy inference: 2596 fps total: 1658 epoch: 3/1000
fps step: 3609 fps step and policy inference: 2601 fps total: 1652 epoch: 4/1000
fps step: 3793 fps step and policy inference: 2693 fps total: 1686 epoch: 5/1000
fps step: 3753 fps step and policy inference: 2714 fps total: 1682 epoch: 6/1000
fps step: 3488 fps step and policy inference: 2510 fps total: 1650 epoch: 7/1000
fps step: 3795 fps step and policy inference: 2716 fps total: 1692 epoch: 8/1000
fps step: 3743 fps step and policy inference: 2707 fps total: 1701 epoch: 9/1000
fps step: 3750 fps step and policy inference: 2720 fps total: 1708 epoch: 10/1000
saving next best rewards:  [23.615122]
=> saving checkpoint 'runs/cartpole_onnx/nn/cartpole_vel_info.pth'
fps step: 3862 fps step and policy inference: 2770 fps total: 1722 epoch: 11/1000
fps

fps step: 3226 fps step and policy inference: 2407 fps total: 1483 epoch: 54/1000
saving next best rewards:  [95.85454]
=> saving checkpoint 'runs/cartpole_onnx/nn/cartpole_vel_info.pth'
fps step: 3502 fps step and policy inference: 2593 fps total: 1583 epoch: 55/1000
fps step: 3397 fps step and policy inference: 2531 fps total: 1661 epoch: 56/1000
fps step: 3361 fps step and policy inference: 2485 fps total: 1603 epoch: 57/1000
fps step: 3726 fps step and policy inference: 2728 fps total: 1707 epoch: 58/1000
fps step: 3649 fps step and policy inference: 2648 fps total: 1687 epoch: 59/1000
fps step: 3738 fps step and policy inference: 2725 fps total: 1726 epoch: 60/1000
saving next best rewards:  [96.303566]
=> saving checkpoint 'runs/cartpole_onnx/nn/cartpole_vel_info.pth'
fps step: 3818 fps step and policy inference: 2818 fps total: 1752 epoch: 61/1000
fps step: 3581 fps step and policy inference: 2595 fps total: 1646 epoch: 62/1000
fps step: 3717 fps step and policy inference: 2714 

In [ ]:
class ModelWrapper(torch.nn.Module):
    '''
    Main idea is to ignore outputs which we don't need from model
    '''
    def __init__(self, model):
        torch.nn.Module.__init__(self)
        self._model = model
        
        
    def forward(self,input_dict):
        input_dict['obs'] = self._model.norm_obs(input_dict['obs'])
        '''
        just model export doesn't work. Looks like onnx issue with torch distributions
        thats why we are exporting only neural network
        '''
        #print(input_dict)
        #output_dict = self._model.a2c_network(input_dict)
        #input_dict['is_train'] = False
        #return output_dict['logits'], output_dict['values']
        return self._model.a2c_network(input_dict)

In [ ]:
agent = runner.create_player()
agent.restore('runs/cartpole_onnx/nn/cartpole_vel_info.pth')

import rl_games.algos_torch.flatten as flatten
inputs = {
    'obs' : torch.zeros((1,) + agent.obs_shape).to(agent.device),
    'rnn_states' : agent.states,
}

with torch.no_grad():
    adapter = flatten.TracingAdapter(ModelWrapper(agent.model), inputs,allow_non_tensor=True)
    traced = torch.jit.trace(adapter, adapter.flattened_inputs,check_trace=False)
    flattened_outputs = traced(*adapter.flattened_inputs)
    print(flattened_outputs)
    
torch.onnx.export(traced, *adapter.flattened_inputs, "cartpole.onnx", verbose=True, input_names=['obs'], output_names=['logits', 'value'])

onnx_model = onnx.load("cartpole.onnx")

# Check that the model is well formed
onnx.checker.check_model(onnx_model)

In [ ]:
ort_model = ort.InferenceSession("cartpole.onnx")

outputs = ort_model.run(
    None,
    {"obs": np.zeros((1, 4)).astype(np.float32)},
)
print(outputs)

In [ ]:
is_done = False
env = agent.env
obs = env.reset()
#prev_screen = env.render(mode='rgb_array')
#plt.imshow(prev_screen)
total_reward = 0
num_steps = 0
while not is_done:
    outputs = ort_model.run(None, {"obs": np.expand_dims(obs, axis=0).astype(np.float32)},)
    action = np.argmax(outputs[0])
    obs, reward, done, info = env.step(action)
    total_reward += reward
    num_steps += 1
    is_done = done
    screen = env.render(mode='rgb_array')
    #plt.imshow(screen)
    #display.display(plt.gcf())    
    #display.clear_output(wait=True)
print(total_reward, num_steps)
#ipythondisplay.clear_output(wait=True)